<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Chroma_DB_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Indexing Documents with Langchain Utilities in Chroma DB
2.   Retrieving Semantically Similar Documents for a Specific Query
3.   Persistence in Chroma DB
4.   Integrating Chroma DB with LLM (OpenAI Chat Models)
5.   Using Question-Answering Chain to Extract Answers from Documents
6.   Utilizing RetrieverQA Chain

Youtube Video : https://youtu.be/5NG8mefEsCU

In [ ]:
!pip install  openai langchain sentence_transformers -q
!pip install chromadb -q

In [ ]:
!pip install unstructured -q

Files Used : https://github.com/PradipNichite/Youtube-Tutorials/tree/main/chroma_db/pets

In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/pets'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


5

https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

5


In [ ]:
# # import openai
# from langchain.embeddings.openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model_name="ada")
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

In [ ]:
query = "What are the different kinds of pets people commonly own?"
matching_docs = db.similarity_search(query)

In [ ]:
matching_docs[0]

Document(page_content='Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.', metadata={'source': '/content/pets/Different Types of Pet Animals.txt'})

In [ ]:
print(matching_docs[0].page_content)

Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.


In [ ]:
matching_docs = db.similarity_search_with_score(query,k=2)
matching_docs

[(Document(page_content='Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.', metadata={'source': '/content/pets/Different Types of Pet Animals.txt'}),
  0.7325009703636169),
 (Document(page_content='Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.', metadata={'source': '/content/pet

Persist a ChromaDB instance

In [ ]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

In [ ]:
vectordb.persist()

In [ ]:
new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
matching_docs = new_db.similarity_search_with_score(query)
matching_docs[0]

(Document(page_content='Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.', metadata={'source': '/content/pets/Different Types of Pet Animals.txt'}),
 0.7325009703636169)

##LLM

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-b..."

In [ ]:
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

###Document QA

https://python.langchain.com/docs/modules/chains/additional/question_answering

https://python.langchain.com/docs/modules/chains/document/

In [ ]:
from langchain.chains.question_answering import load_qa_chain
# chain = load_qa_chain(llm, chain_type="stuff")
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

In [ ]:
query = "What are the emotional benefits of owning a pet?"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Pets offer more than just companionship; they provide emotional support, reduce stress, and can even help their owners lead healthier lives. The bond between pets and their owners is strong, and many people consider their pets as part of the family. This bond can be especially important in times of personal or societal stress, providing comfort and consistency.

Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make 

"Owning a pet can provide emotional support, reduce stress and anxiety, and can even help their owners lead healthier lives. Pets are known to offer companionship, loyalty, and comfort, and many people consider their pets as part of the family. The bond between pets and their owners can be especially important in times of personal or societal stress, providing comfort and consistency. Overall, owning a pet can have a positive impact on one's mental health and well-being."

### Retrieval QA

In [ ]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'Owning a pet can provide emotional support and reduce stress. Pets can also offer comfort and consistency in times of personal or societal stress. Additionally, many people consider their pets as part of the family, which can create a strong bond between pets and their owners.'